In [15]:
%load_ext autoreload
%autoreload 2

from mint.model.transformer import Transformer
from mint.translator import Translator, BeamSearch, GreedySearch
from mint.tokenizer import Tokenizer
from mint.dataset import Dataset
from mint.metrics import bleu, chrf2
from tqdm import tqdm
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
model = Transformer.load("../pretrained/en_sk_small")
source_tokenizer = Tokenizer.load("../tokenizers/en/")
target_tokenizer = Tokenizer.load("../tokenizers/sk/")
dataset = Dataset.load("../datasets/en_sk/")["test"]

MAX_STEPS = 500
USE_CUDA = True

device = torch.device("cuda" if torch.cuda.is_available() and USE_CUDA else "cpu")
if USE_CUDA:
    model.to(device)

def evaluate(translate, max_length=None):
    bleu_total, chrf2_total = 0, 0
    for i, sample in tqdm(enumerate(dataset), "Evaluating", total=MAX_STEPS):
        if i >= MAX_STEPS:
            break
        source = sample["source"]
        target = sample["target"]
        if max_length is not None:
            source = source_tokenizer.clip([source], max_length)[0]
            target = target_tokenizer.clip([target], max_length)[0]
        
        prediction = translate(source) 
        print(f"Source: {source}")
        print(f"Target: {target}")
        print(f"Prediction: {prediction}")
        bleu_total += bleu(prediction, target)
        chrf2_total += chrf2(prediction, target)
        
    bleu_score = bleu_total / MAX_STEPS
    chrf2_score = chrf2_total / MAX_STEPS
    print(f"BLEU: {bleu_score:.4f}, CHRF2: {chrf2_score:.4f}")
    
    return bleu_score, chrf2_score

{'n_blocks': 6, 'vocab_size': 10001, 'transformer_block_config': {'d_model': 256, 'd_feedforward': 1024, 'p_dropout': 0.1, 'attention_config': {'n_heads': 16, 'd_model': 256, 'max_seq_len': 129, 'context_window': None}}, 'embedding_config': {'vocab_size': 10001, 'd_model': 256, 'max_seq_len': 129, 'learnable_positional_embeddings': True}}


In [17]:
translator = Translator(model, source_tokenizer, target_tokenizer)
evaluate(
    lambda x: translator.translate(x, max_length=128),
    max_length=128,
)

Evaluating:   0%|          | 2/500 [00:00<00:37, 13.32it/s]

Source: I invite you to pray for them and for those who have lost their lives.”
Target: Pozývam vás k modlitbe za nich, ako aj za tých, čo stratili život.“
Prediction: Melm, aby som sa za nich modlil a za tých, ktorí stratili ich život."
Source: Thus God's Son came down from heaven.
Target: A vtedy Boh Syn sám zostúpil z výšky nebeskej.
Prediction: A tak aj od Boha, bol to z neba,
Source: When the apostle heard what she had said he said, "Who will rid me of Marwan's daughter?"
Target: Keď apoštol počul, čo povedala, povedal, "Kto ma zbaví Marwanovej dcéry?"
Prediction: Keď sa,, keď je to, čo povedal,, povedal: Kto sa obviňa o marciovej dcére?


Evaluating:   1%|          | 6/500 [00:00<00:38, 12.97it/s]

Source: It is the fruit of the Word delivered to God’s people through His chosen shepherds.
Target: Je to ovocím Slova, ktoré je dané Božiemu ľudu prostredníctvom Jeho vyvolených pastierov.
Prediction: Je to Božia milosť, ktorú sa Božiemu vedzaniu, skrze jeho uctieč.
Source: The atmosphere is filled with God’s presence.
Target: Atmosféra sa začala napĺňať Božou prítomnosťou.
Prediction: Srdcia je však zjavná Božia prítomnosť.
Source: 'Is God willing to prevent evil, but not able?
Target: Chce boh zabrániť zlu, ale nedokáže?
Prediction: A Boh bol ochotný zabrániť zlatu, ale nie?
Source: With God's help, you can control the tongue.
Target: S Božou pomocou dokážeš skrotiť jazyk!
Prediction: S Božou pomocou dokážeš ovládať jazyk.


Evaluating:   2%|▏         | 9/500 [00:00<00:32, 15.15it/s]

...

Evaluating:  99%|█████████▉| 497/500 [00:39<00:00, 12.78it/s]

Source: The Comprehensive Economic and Trade Agreement with Canada should not be ratified, as it also contains this kind of investor protection.”
Target: Komplexná hospodárska a obchodná dohoda s Kanadou by nemala byť podpísaná, keďže tiež obsahuje takýto druh ochrany investora.“
Prediction: Rozdelenakonomická a obchodná dohoda s Kanady by nemala byť ratifikované, keďže tiež obsahuje túto druhú ochranu na ochranu investoru.“
Source: “Greece demands and gets a lot of solidarity from the EU.
Target: "Grécko žiada a dostáva veľkú solidaritu z EÚ.
Prediction: "Grécko vyžaduje a dostane veľa solidarity z EÚ.
Source: I believe in God even when He's silent."
Target: Verím v Boha, aj keď mlčí."
Prediction: Ja verím v Boha a aj keď sa prešiel.“
Source: “When you pray, do not be like the hypocrites.”
Target: “ Keď sa modlíte, nebuďte ako pokrytci.
Prediction: "Keď sa modlíte, nebyte byť ako hypiátori."


Evaluating: 100%|██████████| 500/500 [00:39<00:00, 12.53it/s]

Source: Abraham believed God’s promise and God counted it to him as righteousness.
Target: Uveril tomu, čo mu Boh zasľúbil o veľkom národe a Boh mu jeho vieru počítal za spravodlivosť.
Prediction: Všemohúci veril, že Boh je zasľúbený, že sa mu Boh zjavuje a za jeho spravodlivosť.
Source: Thus, human work corresponds with God’s will.
Target: Vtedy sa vôľa človeka zjednocuje s vôľou Božou.
Prediction: A tak aj tak, že človek je podľa Božej vôle.
Source: bet-at-home also believes in fair game.
Target: bet-at-home tiež verí vo fér hru.
Prediction: bet-at-h-ho ver tiež verí v spravodlivá hru.
BLEU: 0.2014, CHRF2: 0.3204


(tensor(0.2014), 0.3204391541230965)

In [18]:
translator = Translator(model, source_tokenizer, target_tokenizer, search_strategy=BeamSearch(20))

evaluate(
    lambda x: translator.translate(x, max_length=128),
    max_length=128,
)


Evaluating:   0%|          | 1/500 [00:00<01:22,  6.04it/s]

Source: I invite you to pray for them and for those who have lost their lives.”
Target: Pozývam vás k modlitbe za nich, ako aj za tých, čo stratili život.“
Prediction: Žím vás, aby ste sa za nich modlili a pre tých, ktorí stratili ich životy.“
Source: Thus God's Son came down from heaven.
Target: A vtedy Boh Syn sám zostúpil z výšky nebeskej.
Prediction: A tak Boží Syn prišiel z neba.


Evaluating:   1%|          | 3/500 [00:00<01:13,  6.79it/s]

Source: When the apostle heard what she had said he said, "Who will rid me of Marwan's daughter?"
Target: Keď apoštol počul, čo povedala, povedal, "Kto ma zbaví Marwanovej dcéry?"
Prediction: Keď to,, čo nepočul, povedal,, povedal: “Kto sa skryte z dcéru Marwana?”


Evaluating:   1%|          | 4/500 [00:02<07:29,  1.10it/s]

...

Evaluating:  99%|█████████▉| 497/500 [05:18<00:00,  4.45it/s]

Source: “When you pray, do not be like the hypocrites.”
Target: “ Keď sa modlíte, nebuďte ako pokrytci.
Prediction: "Keď sa modlíte, nebyte byť ako vinníkov."


Evaluating: 100%|██████████| 500/500 [05:20<00:00,  1.56it/s]

Source: Abraham believed God’s promise and God counted it to him as righteousness.
Target: Uveril tomu, čo mu Boh zasľúbil o veľkom národe a Boh mu jeho vieru počítal za spravodlivosť.
Prediction: Všemohúci prisľúbil, že Boh je zasľúbený, že sa mu Božia Božia vôľa.
Source: Thus, human work corresponds with God’s will.
Target: Vtedy sa vôľa človeka zjednocuje s vôľou Božou.
Prediction: Toto je Božia vôľa s Božou vôľou.
Source: bet-at-home also believes in fair game.
Target: bet-at-home tiež verí vo fér hru.
Prediction: bet-at-h-im verí aj v spravodlivej hry.
BLEU: 0.2288, CHRF2: 0.3548


(tensor(0.2288), 0.35477034661649387)

In [7]:
!pip install easynmt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [19]:
from easynmt import EasyNMT
model = EasyNMT('opus-mt')

evaluate(
    lambda x: model.translate(x, source_lang='en', target_lang='sk'),
    max_length=128,
)

Evaluating:   1%|          | 3/500 [00:01<03:59,  2.07it/s]

Source: I invite you to pray for them and for those who have lost their lives.”
Target: Pozývam vás k modlitbe za nich, ako aj za tých, čo stratili život.“
Prediction: Pozývam vás, aby ste sa modlili za nich a za tých, ktorí prišli o život.
Source: Thus God's Son came down from heaven.
Target: A vtedy Boh Syn sám zostúpil z výšky nebeskej.
Prediction: Teda Boží Syn zostúpil z neba.
Source: When the apostle heard what she had said he said, "Who will rid me of Marwan's daughter?"
Target: Keď apoštol počul, čo povedala, povedal, "Kto ma zbaví Marwanovej dcéry?"
Prediction: A keď počul apoštol, čo povedala, povedal: Kto ma zbaví dcéry Marwanovej?


Evaluating:   1%|▏         | 7/500 [00:02<01:28,  5.58it/s]

Source: It is the fruit of the Word delivered to God’s people through His chosen shepherds.
Target: Je to ovocím Slova, ktoré je dané Božiemu ľudu prostredníctvom Jeho vyvolených pastierov.
Prediction: Je to ovocie Slova dodané Bohu prostredníctvom Jeho vyvolených pastierov.
Source: The atmosphere is filled with God’s presence.
Target: Atmosféra sa začala napĺňať Božou prítomnosťou.
Prediction: Atmosféra je plná prítomnosti Boha.
Source: 'Is God willing to prevent evil, but not able?
Target: Chce boh zabrániť zlu, ale nedokáže?
Prediction: Je Boh ochotný zabrániť zlu, ale nie je schopný?
Source: With God's help, you can control the tongue.
Target: S Božou pomocou dokážeš skrotiť jazyk!
Prediction: S Božou pomocou môžeš ovládať jazyk.


Evaluating:   2%|▏         | 10/500 [00:02<00:56,  8.61it/s]

...

Evaluating: 100%|█████████▉| 498/500 [00:33<00:00, 16.21it/s]

Source: I believe in God even when He's silent."
Target: Verím v Boha, aj keď mlčí."
Prediction: Verím v Boha, aj keď je tichý."
Source: “When you pray, do not be like the hypocrites.”
Target: “ Keď sa modlíte, nebuďte ako pokrytci.
Prediction: Keď sa modlíte, nebuďte ako pokrytci.
Source: Abraham believed God’s promise and God counted it to him as righteousness.
Target: Uveril tomu, čo mu Boh zasľúbil o veľkom národe a Boh mu jeho vieru počítal za spravodlivosť.
Prediction: Abrahám veril sľubu Boha a Boh mu ho považoval za spravodlivosť.
Source: Thus, human work corresponds with God’s will.
Target: Vtedy sa vôľa človeka zjednocuje s vôľou Božou.
Prediction: Teda ľudské dielo zodpovedá Božej vôli.


Evaluating: 100%|██████████| 500/500 [00:33<00:00, 14.95it/s]

Source: bet-at-home also believes in fair game.
Target: bet-at-home tiež verí vo fér hru.
Prediction: bet-at-home tiež verí vo fair game.
BLEU: 0.3151, CHRF2: 0.4731


(tensor(0.3151), 0.47314069245854923)